In [1]:
from datasets import load_dataset
import random
from transformers import pipeline
import torch
from sentence_transformers import SentenceTransformer, util
import pandas as pd
#from tqdm import tqdm
from tqdm.notebook import tqdm
import evaluate

In [2]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
#dataset = load_dataset('code-search-net/code_search_net')
dataset = load_dataset("Fsoft-AIC/the-vault-function", split_set=["test"], trust_remote_code=True)

javascript-00000-of-00001.parquet:   0%|          | 0.00/26.4M [00:00<?, ?B/s]

php-00000-of-00001.parquet:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [18]:
sample_df = pd.DataFrame(dataset['test'])
sample_df.columns

Index(['hexsha', 'repo', 'path', 'license', 'language', 'identifier',
       'return_type', 'original_string', 'original_docstring', 'docstring',
       'docstring_tokens', 'code', 'code_tokens', 'short_docstring',
       'short_docstring_tokens', 'comment', 'parameters', 'docstring_params'],
      dtype='object')

In [ ]:
dataset_sample = sample_df.groupby('language', group_keys=False).sample(n=20)

# Testing the Baseline model

Here we define the system prompt for the Llama 2 model.

In [ ]:
def test_prompt(language):
    return \
    f'''You are a helpful agent designed to simplify code documentation for beginner programmers.
    You will be provided with a block of {language} code and the existing doucmentation that accompanies it.
    Simplify the given documentation, using the provided code as context, so that it is understandable
    to beginner programmers. Output absolutely nothing else besides the simplified documentation.
    Make sure to keep any documentation formatting codes present in the simplified documentation.
    If you feel that the existing documentation is simple enough and meaning would be lost by simplifying
    it further, feel free to keep the documentation as is. Here is the original documentation and code:'''

Creating the pipeline for the Llama 2 model using the HuggingFace transformers library. Modified from the example here: https://huggingface.co/docs/transformers/en/model_doc/llama2

In [22]:
pipe_llama = pipeline(
    task="text-generation",
    model="meta-llama/Llama-2-7b-chat-hf",
    torch_dtype=torch.float16,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

c:\Users\eaden\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\eaden\.cache\huggingface\hub\models--meta-llama--Llama-2-7b-chat-hf. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Device set to use cpu


In [23]:
eval_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\eaden\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\eaden\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
message = [
    {"role": "system", "content": test_prompt(dataset['test'][0]['language'])},
    {"role": "user", "content": f"Documentation:\n{dataset['test'][0]['original_docstring']}\n\nCode:\n{dataset['test'][0]['original_string']}"}
]
print(f"Original Documentation:\n{dataset['test'][0]['original_docstring']}\n")
print(f"Code:\n{dataset['test'][0]['original_string']}\n")
print(pipe(message, pad_token_id=pipe.tokenizer.eos_token_id)[0]['generated_text'][-1]['content'])

Original Documentation:
Function to read registers from MMC or SD card

Code:
static unsigned int
mmc_boot_read_reg(struct mmc_boot_card *card,
		  unsigned int data_len,
		  unsigned int command, unsigned int addr, unsigned int *out)
{
	struct mmc_boot_command cmd;
	unsigned int mmc_ret = MMC_BOOT_E_SUCCESS;
	unsigned int mmc_reg = 0;
	mmc_reg = readl(MMC_BOOT_MCI_CLK);
	mmc_reg |= MMC_BOOT_MCI_CLK_ENA_FLOW;
	writel(mmc_reg, MMC_BOOT_MCI_CLK);
	mmc_reg = 0xFFFFFFFF;
	writel(mmc_reg, MMC_BOOT_MCI_DATA_TIMER);
	writel(data_len, MMC_BOOT_MCI_DATA_LENGTH);
	mmc_reg =
	    MMC_BOOT_MCI_DATA_ENABLE | MMC_BOOT_MCI_DATA_DIR | (data_len <<
								MMC_BOOT_MCI_BLKSIZE_POS);
#if MMC_BOOT_ADM
	mmc_reg |= MMC_BOOT_MCI_DATA_DM_ENABLE;
#endif
	writel(mmc_reg, MMC_BOOT_MCI_DATA_CTL);
	memset((struct mmc_boot_command *)&cmd, 0,
	       sizeof(struct mmc_boot_command));
	cmd.cmd_index = command;
	cmd.argument = addr;
	cmd.cmd_type = MMC_BOOT_CMD_ADDRESS;
	cmd.resp_type = MMC_BOOT_RESP_R1;
	mmc_ret = m

KeyboardInterrupt: 

In [ ]:
llama_semantic_similarities_untrained = []
llama_metrics_untrained = evaluate.combine(['rouge', 'meteor'])

for instance in tqdm(dataset_sample.itertuples()):
    message = [
        {"role": "system", "content": test_prompt(instance.language)},
        {"role": "user", "content": f"Documentation:\n{instance.original_docstring}\n\nCode:\n{instance.original_string}"}
    ]

    result = pipe(message, pad_token_id=pipe_llama.tokenizer.eos_token_id)[0]['generated_text'][-1]['content']

    embedding_original = eval_model.encode(instance.original_docstring, convert_to_tensor=True)
    embedding_predicted = eval_model.encode(result, convert_to_tensor=True)

    llama_semantic_similarities_untrained.append(util.pytorch_cos_sim(embedding_original, embedding_predicted).item())
    llama_metrics_untrained.add(predictions=result, references=instance.original_docstring)

[nltk_data] Downloading package wordnet to /home/adeniji/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/adeniji/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/adeniji/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


  0%|          | 0/50 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
train_df = pd.read_excel("self_training_annotated.xlsx", sheet_name="Sheet2", usecols=[5, 10, 12, 16])
train_df.columns

In [ ]:
def training_prompt(language):
    return \
    f'''You are a helpful agent designed to simplify code documentation for beginner programmers.
    You will be provided with a block of {language} code and the existing doucmentation that accompanies it.
    Additionally you will be passed an example of what the documentation would look like if it was simplified 
    to be understandable to beginner programmers. No output is necessary, you are being trained to understand what 
    simplified documentation should lool like. Here is the original documentation and code:'''

In [ ]:
for instance in tqdm(dataset_sample.itertuples()):
    message = [
        {"role": "system", "content": training_prompt(instance.language)},
        {"role": "user", "content": f"Documentation:\n{instance.original_docstring}\n\nCode:\n{instance.original_string}"}
    ]

    pipe(message, pad_token_id=pipe_llama.tokenizer.eos_token_id)

In [ ]:
llama_semantic_similarities_trained = []
llama_metrics_trained = evaluate.combine(['rouge', 'meteor'])


for instance in tqdm(dataset_sample.itertuples()):
    message = [
        {"role": "system", "content": test_prompt(instance.language)},
        {"role": "user", "content": f"Documentation:\n{instance.original_docstring}\n\nCode:\n{instance.original_string}"}
    ]

    result = pipe(message, pad_token_id=pipe_llama.tokenizer.eos_token_id)[0]['generated_text'][-1]['content']

    embedding_original = eval_model.encode(instance.original_docstring, convert_to_tensor=True)
    embedding_predicted = eval_model.encode(result, convert_to_tensor=True)

    llama_semantic_similarities_trained.append(util.pytorch_cos_sim(embedding_original, embedding_predicted).item())
    llama_metrics_trained.add(predictions=result, references=instance.original_docstring)

# Summary statistics results
Semantic Similarity

In [ ]:
print("Untrained Llama 2: \n")
pd.DataFrame(llama_semantic_similarities_untrained).describe()
print("Trained Llama 2: \n")
pd.DataFrame(llama_semantic_similarities_trained).describe()

ROUGE AND METEOR

In [ ]:
llama_metrics_untrained.compute()
llama_metrics_trained.compute()